# Dataset loading and analysis

In [2]:
#importing the necessary libraries
import pandas as pd 
from datasets import load_dataset 

#loading the dataset using the Hugging Face datasets library
df = load_dataset("google-research-datasets/go_emotions")

In [3]:
# Exploring the training set
df['train'][0]

{'text': "My favourite food is anything I didn't have to cook myself.",
 'labels': [27],
 'id': 'eebbqej'}

In [6]:
df['validation'][0]

{'text': 'Is this in New Orleans?? I really feel like this is New Orleans.',
 'labels': [27],
 'id': 'edgurhb'}

In [ ]:
#mapping the labels to the emotion names; 
# Link to mapping: https://github.com/google-research/google-research/blob/756ae45c4880ad6a01869608250d85a8fb253799/goemotions/data/emotions.txt

label_index = {
    "0": "admiration",
    "1": "amusement",
    "2": "anger",
    "3": "annoyance",
    "4": "approval",
    "5": "caring",
    "6": "confusion",
    "7": "curiosity",
    "8": "desire",
    "9": "disappointment",
    "10": "disapproval",
    "11": "disgust",
    "12": "embarassment",
    "13": "excitement",
    "14": "fear",
    "15": "gratitude",
    "16": "grief",
    "17": "joy",
    "18": "love",
    "19": "nervousness",
    "20": "optimism",
    "21": "pride",
    "22": "realization",
    "23": "relief",
    "24": "remorse",
    "25": "sadness",
    "26": "surprise",
    "27": "neutral"
}

In [31]:
train_subset[998]

{'text': 'Oh god yes. Top quality cringe. Thank you for sharing',
 'labels': [4, 15],
 'id': 'eemffjk'}

In [32]:
#Get the first 1000 rows of the training set, map the labels to the emotion names and store in data_df

# Get first 1000 rows from training set
train_subset = df['train'].select(range(1000))

# Convert to list of dictionaries and map labels to emotion names
data_list = []
for example in train_subset:
    # Map each label (integer) to its emotion name using label_index
    emotion_labels_list = [label_index[str(label)] for label in example['labels']]
    emotion_labels_string = ", ".join(emotion_labels_list)
    data_list.append({
        'comment': example['text'],
        'emotion labels': emotion_labels_string  # Now contains emotion names instead of numbers
    })

# Create DataFrame
comments_df = pd.DataFrame(data_list)


In [48]:
#print the first 10 rows of the dataframe
comments_df.sample(10)

,comment,emotion labels
329,I have dayz but haven’t played in years. Last ...,amusement
763,Is that the dude from spiderman?,curiosity
647,"Wow this was an eye-opening response, and you’...","admiration, approval, surprise"
237,I wanted to see him show out over Kemba.,"desire, neutral"
401,[NAME] looked terrible that game. I was really...,fear
409,"Same here man, terrible feeling",fear
954,"Stop breaking the rules, you're still posting ...",neutral
0,My favourite food is anything I didn't have to...,neutral
889,I've used tofu and nutritional yeast before. I...,approval
968,deliberately getting in someone's face while t...,neutral


In [ ]:
# Count all individual emotions in the dataframe
# quick check to see if all emotions are used

emotion_counts = {emotion: 0 for emotion in label_index.values()}

for emotion_combo in comments_df['emotion labels']:
    for emotion in emotion_combo.split(', '):
        emotion_counts[emotion] += 1

# Print the counts
print("=" * 60)
print("COUNT OF ALL EMOTIONS IN THE DATAFRAME:")
print("=" * 60)
for emotion, count in sorted(emotion_counts.items()):
    print(f"{emotion}: {count}")

# (2) Check for missing emotions
missing_emotions = [emotion for emotion, count in emotion_counts.items() if count == 0]

print("\n" + "=" * 60)
print("MISSING EMOTIONS:")
print("=" * 60)
if not missing_emotions:
    print("All emotions are present at least once.")
else:
    print("The following emotions are missing:", missing_emotions)
print("=" * 60)



COUNT OF ALL EMOTIONS IN THE DATAFRAME:
admiration: 99
amusement: 61
anger: 41
annoyance: 57
approval: 67
caring: 21
confusion: 33
curiosity: 50
desire: 13
disappointment: 34
disapproval: 49
disgust: 13
embarassment: 6
excitement: 13
fear: 10
gratitude: 63
grief: 6
joy: 39
love: 47
nervousness: 3
neutral: 336
optimism: 29
pride: 1
realization: 19
relief: 2
remorse: 9
sadness: 32
surprise: 35

MISSING EMOTIONS:
All emotions are present at least once.


# LLM setup for performance comparison

In [ ]:
# -----------------------------
# Import + API key setup block
# -----------------------------
import os
from dotenv import load_dotenv

# Load variables from the .env file (including OPENAI_API_KEY)
load_dotenv()

# Set your API key; Ensure that you have created an API key in Open AI and added it to the .env file
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# LangChain core utilities
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# OpenAI Chat models via LangChain
from langchain_openai import ChatOpenAI

# Progress bar for batch inference
from tqdm import tqdm

In [80]:
#Create a prompt template to pass to the LLM for inference
prompt_template = """
    You are a highly intelligent emotion classification assistant.
    You carefully read a comment, and label it with one or more pre-selected emotion labels to it.
    The emotion labels are listed here:
    {emotions} 
    Your output should be just the emotion label that you are applying to the comment, and if you
    think there are multple labels then output the labels separated by commas.
    The comment to analyze is here: {comment}
    """
#initialize the LangChain string output parser
output_parser = StrOutputParser()

#Initialize the OpenAI model for comparison - let's use gpt-4o-mini 
gpt4o_mini_llm = ChatOpenAI(temperature = 0.0, model = "gpt-4o-mini")  

#Initialize LangChain prompt template object 
emotion_labeling_prompt = ChatPromptTemplate.from_template(prompt_template) 

#Create an inference chain 
labeling_chain = ( 

    {"comment" : RunnablePassthrough(), "emotions" : RunnablePassthrough()}
    | emotion_labeling_prompt
    | gpt4o_mini_llm
    | output_parser
)

#create a list of emotions to pass to the LLM in the prompt 
emotion_labels = []
for label in label_index.values():
    print(label) 
    emotion_labels.append(label)

admiration
amusement
anger
annoyance
approval
caring
confusion
curiosity
desire
disappointment
disapproval
disgust
embarassment
excitement
fear
gratitude
grief
joy
love
nervousness
optimism
pride
realization
relief
remorse
sadness
surprise
neutral


# GPT 4-o mini Comparison

In [ ]:
# The initial gpt-4-t baseline on the first 100 comments
def analyze_emotion(comment, emotions):
    input_data = {
        "comment": comment,
        "emotions": emotion_labels
    }
    try:
        result = labeling_chain.invoke(input_data)
        return result
    except Exception as e:
        return str(e)

gpt4o_mini_emotion_labels = []

# Loop through the first 100 comments with progress bar
for i in tqdm(range(100), desc="Processing comments", unit="comment"):
    comment = comments_df['comment'][i]  # Get the i-th comment
    emotion_label = analyze_emotion(comment, emotion_labels)  # Analyze the comment
    gpt4o_mini_emotion_labels.append(emotion_label)  # Append the result to the list

# Assign the list of emotion labels to a new column in the DataFrame
# Assign directly to first 100 rows using .loc to avoid NaN values in remaining rows
comments_df.loc[:99, 'gpt4o-mini emotion label'] = gpt4o_mini_emotion_labels

In [84]:
comments_df[comments_df['gpt4o-mini emotion label'].isna()]

,comment,emotion labels,gpt4t emotion label,gpt4o-mini emotion label
100,You just make her sound awesome.,admiration,NaN,NaN
101,seriously wtf. I want to see how the whole han...,annoyance,NaN,NaN
102,Your team lacks ethics.,disapproval,NaN,NaN
103,"[NAME] was useless, aside from bandaging [NAME...",neutral,NaN,NaN
104,"Not the **emotional effect**, when it's in the...",neutral,NaN,NaN
...,...,...,...,...
995,I love [NAME],love,NaN,NaN
996,Woman here - $50 per orgasm please! I could ea...,optimism,NaN,NaN
997,A royal with creme,neutral,NaN,NaN
998,Oh god yes. Top quality cringe. Thank you for ...,"approval, gratitude",NaN,NaN
